# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data_preprocess import series_to_supervised,load_data


import numpy as np

In [2]:
r='data/instant_data/rain_small.csv'
w='data/instant_data/water_small.csv'
#r = "data/instant_data/rain.csv"
#w = 'data/instant_data/water.csv'
rw = load_data(r,w)
df =rw.df.resample('d').mean()

DataFrame shape: (465169, 35)


In [3]:
TARGET = 'CPY015_w'

## Period 2013-2015 ##
X_in = df['2013-01-01':'2015-01-01'].interpolate(limit=15)
X_in = X_in.astype('float32')
X = X_in.drop([TARGET],axis=1)
xlabels = list(X.columns)


# REPLACE NAN WITH 0
X = X.fillna(0).values
y = np.log(X_in[TARGET].fillna(0).values)
where_are_NaNs = np.isnan(y)
y[where_are_NaNs] = 0


<ipython-input-3-260aebac4cf6>:12: RuntimeWarning: invalid value encountered in log
  y = np.log(X_in[TARGET].fillna(0).values)


# MARS feature selection (from all stations)

In [4]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.296832  1      0.298  0.000  0.000  
1     0       23   424   0.183515  3      0.187  0.382  0.373  
2     0       21   255   0.161118  5      0.166  0.457  0.442  
3     0       25   545   0.153550  7      0.160  0.483  0.461  
4     0       20   348   0.150323  9      0.159  0.494  0.465  
5     0       24   331   0.148433  11     0.160  0.500  0.464  
6     0       22   238   0.145528  13     0.159  0.510  0.467  
7     0       7    -1    0.143761  14     0.158  0.516  0.469  
8     0       24   568   0.142312  16     0.159  0.521  0.467  
9     0       16   204   0.140910  18     0.159  0.525  0.465  
10    0       31   -1    0.139470  19     0.159  0.530  0.466  
11    0       31   315   0.138801  21     0.160  0.532  0.461  
12    0       31 

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:813: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  pruning_passer.run()


--------------------------------------------
iter  bf  terms  mse   gcv    rsq    grsq   
--------------------------------------------
0     -   56     0.13  0.192  0.575  0.355  
1     38  55     0.13  0.190  0.575  0.361  
2     44  54     0.13  0.189  0.575  0.366  
3     18  53     0.13  0.187  0.575  0.371  
4     21  52     0.13  0.186  0.575  0.377  
5     50  51     0.13  0.184  0.576  0.382  
6     25  50     0.13  0.182  0.575  0.387  
7     37  49     0.13  0.181  0.575  0.392  
8     29  48     0.13  0.180  0.575  0.397  
9     46  47     0.13  0.178  0.575  0.402  
10    9   46     0.13  0.177  0.576  0.407  
11    33  45     0.13  0.175  0.575  0.411  
12    53  44     0.13  0.174  0.575  0.416  
13    23  43     0.13  0.172  0.575  0.421  
14    49  42     0.13  0.171  0.575  0.425  
15    27  41     0.13  0.169  0.576  0.431  
16    14  40     0.13  0.168  0.577  0.437  
17    31  39     0.13  0.166  0.577  0.441  
18    16  38     0.13  0.166  0.574  0.442  
19    52  

/home/song/anaconda3/envs/tf/lib/python3.8/site-packages/pyearth/earth.py:1066: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid = np.linalg.lstsq(B, weighted_y[:, i])[0:2]


Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [5]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [6]:
print(model.summary_feature_importances(sort_by='gcv')[:2000])

             rss    gcv    nb_subsets
CPY009_w     0.58   0.72   0.04          
CPY004_w     0.15   0.17   0.06          
CPY003_w     0.08   0.07   0.07          
THA001_w     0.03   0.02   0.04          
KWN002_r     0.01   0.01   0.02          
CHM006_w     0.02   0.01   0.04          
NAN013_r     0.00   0.00   0.00          
NAN012_r     0.00   0.00   0.00          
NAN011_r     0.00   0.00   0.00          
NAN010_r     0.00   0.00   0.00          
NAN006_r     0.00   0.00   0.00          
PIN003_r     0.00   0.00   0.00          
CPY015_r     0.00   0.00   0.00          
CPY010_r     0.00   0.00   0.00          
CPY009_r     0.00   0.00   0.00          
CPY004_r     0.01   0.00   0.04          
CHM006_r     0.00   0.00   0.00          
CHM004_r     0.00   0.00   0.00          
PIN001_r     0.00   0.00   0.00          
PIN005_r     0.01   0.00   0.04          
PIN004_r     0.00   0.00   0.00          
PIN004_w     0.00   0.00   0.00          
THA001_r     0.00   0.00   0.00       

# MARS feature selection (Times interval perspective)

In [7]:
n_timelag=7

reframed_data = series_to_supervised(X_in.iloc[:20,:], n_in= 3,n_out=1)
# not work
X_in.shape,reframed_data.shape

((731, 35), (0, 140))

In [8]:
def to_sub(df,n_in=5,n_out=3):
    df_in = df.shift(n_in)
    df_out = df.shift(-n_out)
    return df_in

In [9]:
test = X_in.iloc[:,:5]

print(test.head())
print(to_sub(test).head())

            CHM002_r  CHM004_r  CHM006_r  CPY004_r  CPY009_r
date                                                        
2013-01-01       0.0       NaN       NaN       0.0       0.0
2013-01-02       0.0       NaN       NaN       0.0       0.0
2013-01-03       0.0       NaN       NaN       0.0       0.0
2013-01-04       0.0       NaN       NaN       0.0       0.0
2013-01-05       0.0       NaN       NaN       0.0       0.0
            CHM002_r  CHM004_r  CHM006_r  CPY004_r  CPY009_r
date                                                        
2013-01-01       NaN       NaN       NaN       NaN       NaN
2013-01-02       NaN       NaN       NaN       NaN       NaN
2013-01-03       NaN       NaN       NaN       NaN       NaN
2013-01-04       NaN       NaN       NaN       NaN       NaN
2013-01-05       NaN       NaN       NaN       NaN       NaN
